In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [22]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [23]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model = "qwen/qwen3-next-80b-a3b-instruct")
mini_model = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

class EmailState(AgentState):
    email: str

agent = create_agent(
    model=model,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [24]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [25]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John, '
                                                                          'no '
                                                                          'problem, '
                                                                          "let's "
                                                                          'reschedule. '
                                                                          'Please '
                                                                          'suggest '
                                                                          'a '
                                                                          'new '
                                                                          'time '
                                                                          'that '
      

In [26]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': "Hi John, no problem, let's reschedule. Please suggest a new time that works for you. Best, Seán"}, 'description': 'Tool execution requires approval\n\nTool: send_email\nArgs: {\'body\': "Hi John, no problem, let\'s reschedule. Please suggest a new time that works for you. Best, Seán"}'}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='03d37923eb05f9b77c71c99d35834416')]


In [27]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John, no problem, let's reschedule. Please suggest a new time that works for you. Best, Seán


## Approve

In [8]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='2d84483f-a595-41ce-bbb7-dde2034d9adc'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_36fb8527bf9049fdb8b762d3', 'index': -1, 'type': 'function', 'function': {'name': 'read_email', 'arguments': '{}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'reasoning_content': None, 'tool_calls': [{'id': 'call_36fb8527bf9049fdb8b762d3', 'index': -1, 'type': 'function', 'function': {'name': 'read_email', 'arguments': '{}'}}], 'token_usage': {'prompt_tokens': 204, 'total_tokens': 220, 'completion_tokens': 16, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'finish_reason': 'tool_calls', 'model_name': 'qwen/qwen3-next-80b-a3b-instruct'}, id='lc_run--019c9

## Reject

In [14]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John, '
                                                                          'no '
                                                                          'problem, '
                                                                          "let's "
                                                                          'reschedule. '
                                                                          'Please '
                                                                          'suggest '
                                                                          'a '
                                                                          'new '
                                                                          'time '
                                                                          'that '
      

In [15]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John, no problem, let's reschedule. Please suggest a new time that works for you. Your merciful leader, Seán.


## Edit

In [28]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='94e05dae-2871-423c-b7e2-4e9197e51831'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dad69e5ef3a847e896dbd365', 'index': -1, 'type': 'function', 'function': {'name': 'read_email', 'arguments': '{}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'reasoning_content': None, 'tool_calls': [{'id': 'call_dad69e5ef3a847e896dbd365', 'index': -1, 'type': 'function', 'function': {'name': 'read_email', 'arguments': '{}'}}], 'token_usage': {'prompt_tokens': 204, 'total_tokens': 220, 'completion_tokens': 16, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'finish_reason': 'tool_calls', 'model_name': 'qwen/qwen3-next-80b-a3b-instruct'}, id='lc_run--019c9